In [58]:
from os import walk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import re
from sklearn.naive_bayes import BernoulliNB

In [2]:
X = []
Y = []

In [3]:
def setData(file):
    header = file.readline()
    empty_row = file.readline()
    message = file.readline()
    X.append(np.array(" ".join(header.split()[1:]) + " " + message).astype(str))

In [4]:
def setTarget(file):
    legit_mark = re.search("^.*legit.*$", file)
    if(legit_mark):
        Y.append(-1)
    else:
        Y.append(1)

In [5]:
def parseMessage(filepath):
    with open(filepath, "r") as file:
        setTarget(filepath)
        setData(file)

In [6]:
for n in range(1, 11):
    for (dirpath, dirnames, filenames) in walk("./messages/part" + str(n) +"/"):
        for filename in filenames:
            parseMessage("./messages/part" + str(n) +"/" + filename)
        break

In [7]:
Y = np.array(Y)
X = np.array(X)

In [8]:
X_transformed = CountVectorizer().fit_transform(X).toarray()

In [9]:
pd.DataFrame(X_transformed, columns = CountVectorizer().fit(X).get_feature_names())

,10,100,1000,10000,10001,10002,10003,10004,10005,10006,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1086,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,2,0,0
1087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1088,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
fog_gc = []

In [161]:
for n in range(1, 3):
    tmp_alpha1 = np.arange(1.2, 5, 0.1).reshape(-1, 1)
    tmp_alpha2 = np.array([n] * len(tmp_alpha1)).reshape(-1, 1)
    
    fog_gc = fog_gc + np.append(tmp_alpha2, tmp_alpha1, axis = 1).tolist()

In [163]:
parameters_grid = {
    'class_prior': fog_gc,
    'alpha':  np.arange(0.00001, 0.005, 10)
}

In [164]:
sv = GridSearchCV(BernoulliNB(), parameters_grid, scoring="precision", cv=10).fit(X_transformed, Y)

In [165]:
sv.best_estimator_
#BernoulliNB(alpha=0.01, binarize=0.0, class_prior=[1.0, 1.5], fit_prior=True)

BernoulliNB(alpha=1e-05, binarize=0.0, class_prior=[1.0, 1.2], fit_prior=True)

In [166]:
sv.best_score_

0.9871671722278396

In [167]:
from sklearn.metrics import confusion_matrix

clf1 = sv.best_estimator_
print(confusion_matrix(Y, clf1.fit(X_transformed, Y).predict(X_transformed)))

[[610   0]
 [  5 475]]
